<a href="https://colab.research.google.com/github/meredityman/aiartathon-II-volumetric-ai-workshop/blob/main/GeneratingDatasets/Collab/Notebook_StyleGan2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Training StyleGAN2 on Colab
Y’all won’t stop asking me about this so here ya go 😂

If it were me I’d sign up for Colab Pro ($10/month) to get a couple extra hours of training time in per session. But you crazy kids can do whatever you want.

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Oct 31 15:45:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

##Mounting Google Drive
So I’m actually gonna install my entire repo directly into Google Drive. This will make the setup a little easier, but its a little strange I admit.

First, connect your Drive to Colab.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Install the repo
**Only do this for the first time ever setting this up!**

If this is your first time ever running this notebook, you’ll want to install my fork of StyleGAN2 to your Drive account. Make sure you have ample space on your Drive (I’d say at least 50GB). This will install the repo and add some dependecies (like transferring from FFHQ the first time).

Every time after your first use of this notebook you’ll want to skip this cell and run the cell after this.

In [4]:
#SKIP this if you already have a stylegan2 folder in your google drive
%cd /content/drive/My\ Drive/
!mkdir stylegan2-colab
%cd stylegan2-colab/
!git clone https://github.com/dvschultz/stylegan2
%cd stylegan2
!mkdir pkl
%cd pkl
!gdown --id 1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF #inception: https://drive.google.com/open?id=1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF
%cd ../
!mkdir results
!mkdir results/00001-pretrained
%cd results/00001-pretrained
!gdown --id 1UlDmJVLLnBD9SnLSMXeiZRO6g-OMQCA_
!mv stylegan2-ffhq-config-f.pkl network-snapshot-10000.pkl
%cd ../../
%mkdir datasets

/content/drive/My Drive
mkdir: cannot create directory ‘stylegan2-colab’: File exists
/content/drive/My Drive/stylegan2-colab
fatal: destination path 'stylegan2' already exists and is not an empty directory.
/content/drive/My Drive/stylegan2-colab/stylegan2
mkdir: cannot create directory ‘pkl’: File exists
/content/drive/My Drive/stylegan2-colab/stylegan2/pkl
Downloading...
From: https://drive.google.com/uc?id=1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF
To: /content/drive/My Drive/stylegan2-colab/stylegan2/pkl/inception_v3_features.pkl
100% 87.3M/87.3M [00:01<00:00, 46.7MB/s]
/content/drive/My Drive/stylegan2-colab/stylegan2
mkdir: cannot create directory ‘results’: File exists
mkdir: cannot create directory ‘results/00001-pretrained’: File exists
/content/drive/My Drive/stylegan2-colab/stylegan2/results/00001-pretrained
Downloading...
From: https://drive.google.com/uc?id=1UlDmJVLLnBD9SnLSMXeiZRO6g-OMQCA_
To: /content/drive/My Drive/stylegan2-colab/stylegan2/results/00001-pretrained/stylegan2-ff

##Picking up from a previous session
If you already have the StyleGAN2 repo installed in Drive skip the above cell and run the following. This will make sure you have the latest version in case I make updates.

In [5]:
#USE this if you already have a stylegan2 folder in google drive
%cd /content/drive/My\ Drive/stylegan2-colab/stylegan2
!git pull

/content/drive/My Drive/stylegan2-colab/stylegan2
Already up to date.


##Make sure Tensorflow 1.15 is set
Colab now defaults to Tensorflow 2. Make sure you run this cell to reset it to TF1.

In [6]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


##Converting your dataset
StyleGAN requires you to convert your standard jpg or png images into a new format (.tfrecords). 

I’ve seen some recommendations to run this command every time you restart your Colab machine. I think if you ahve a small-ish dataset (< 2000 images) that’s probably unnecessary.

I recommend you upload your dataset to Google Drive and copy its path from the Drive folder in Colab and paste its path in the below cell.

After the `create_from_images` argument you need to pass in two paths. The first path is where the .tfrecords files should be output (just edit the last part to have a unique name). The second path is to the directory of your images.




In [7]:
#2nd argument is where to put your tfrecords files
#3rd should point at your image dataset
raw_data_path = '"./datasets/trees-raw/Renders"'
!python dataset_tool.py create_from_images ./datasets/trees $raw_data_path

Loading images from "./datasets/trees-raw/Renders"
  0% 0/800 [00:00<?, ?it/s]dataset_tool.py:87: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
100% 800/800 [04:41<00:00,  2.84it/s]


##Training
Note: this will require you to restart your Colab machine every 8–16 hours. You’ve been warned!

This library is set up to automatically find your latest .pkl file so it should keep re-training and making progress.

##Training Options
`--dataset`

This should be the name you used in the first path when converting your dataset.

`--mirror-augment`

Using this option loads some images at random mirrored horizontally (left-to-right). This might help if you have a very small dataset.

`--metrics`

METRICS DON’T MATTER. It’s art! Use your eyes. Set `--metrics=None` and live your life.

If you must use metrics, you have a few options. `fid50k`, the default, uses Frechet Inception Distance score. It’s what was used in StyleGAN1 and what most people know. It’s fine for images of animals and things, but it’s not great. `ppl_wend` is what StyleGAN2 prefers and claims to be more accurate. There are a bunch of other options but I’d recommend you stick with those. Note that both of these take 30–45minutes to run every time it runs so that cuts into your training time in Colab.


In [8]:
!nvcc test_nvcc.cu -o test_nvcc -run
!nvidia-smi -L

CPU says hello.
GPU says hello.
GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-584a5b7d-12b0-716b-e606-96e407b2dd46)


In [ ]:
!python run_training.py --num-gpus=1 --data-dir=./datasets --config=config-f --dataset=trees --mirror-augment=true --metrics=None --resume_pkl='none'

Couldn't find valid snapshot, starting over
Local submit - run_dir: results/00007-stylegan2-trees-1gpu-config-f
dnnlib: Running training.training_loop.training_loop() on localhost...
Streaming data using training.dataset.TFRecordDataset...
tcmalloc: large alloc 4294967296 bytes == 0x55fb39372000 @  0x7f6bb6ffd001 0x7f6bb3ae154f 0x7f6bb3b31b58 0x7f6bb3b35b17 0x7f6bb3bd4203 0x55fb31f87544 0x55fb31f87240 0x55fb31ffb627 0x55fb31ff5ced 0x55fb31f8948c 0x55fb31fca159 0x55fb31fc70a4 0x55fb31f89698 0x55fb31ff7fe4 0x55fb31ff59ee 0x55fb31ec7e2b 0x55fb31ff7fe4 0x55fb31ff59ee 0x55fb31ec7e2b 0x55fb31ff7fe4 0x55fb31f88afa 0x55fb31ff6915 0x55fb31f88afa 0x55fb31ff6c0d 0x55fb31ff59ee 0x55fb31ec7e2b 0x55fb31ff7fe4 0x55fb31ff59ee 0x55fb31ec7e2b 0x55fb31ff7fe4 0x55fb31f88afa
tcmalloc: large alloc 4294967296 bytes == 0x55fc39372000 @  0x7f6bb6ffb1e7 0x7f6bb3ae146e 0x7f6bb3b31c7b 0x7f6bb3b3235f 0x7f6bb3bd4103 0x55fb31f87544 0x55fb31f87240 0x55fb31ffb627 0x55fb31ff59ee 0x55fb31f88bda 0x55fb31ff7737 0x55fb31ff

Once running, your training files will show up in the results folder.

#Testing the model (generating images)
The following command will generate 55 sample images from the model.

##Options
`--network`

Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).

`--seeds`

This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation.

`--truncation-psi`

Truncation is a special argument of StyleGAN. Essentially values that are closer to 0 will be more real than numbers further away from 0. I generally recommend a value between `0.5` and `1.0`. `0.5` will give you pretty "realistic" results, while `1.0` is likely to give you "weirder" results.

In [10]:
!python run_generator.py generate-images --network=/content/ladiesfloralcrop-network-snapshot-010237.pkl --seeds=3875451-3876000 --truncation-psi=0.7

Traceback (most recent call last):
  File "run_generator.py", line 16, in <module>
    from opensimplex import OpenSimplex
ModuleNotFoundError: No module named 'opensimplex'


Let’s zip the generated files and download them.

In [11]:
!zip -r generated-0.7.zip /content/stylegan2/results/00000-generate-images

	zip warning: name not matched: /content/stylegan2/results/00000-generate-images

zip error: Nothing to do! (try: zip -r generated-0.7.zip . -i /content/stylegan2/results/00000-generate-images)


##Interpolation


In [12]:
!python run_generator.py generate-latent-walk --network=/content/ladiesfloralcrop-network-snapshot-010237.pkl --seeds=3,11,17,25,3 --frames 200 --truncation-psi=0.7

Traceback (most recent call last):
  File "run_generator.py", line 16, in <module>
    from opensimplex import OpenSimplex
ModuleNotFoundError: No module named 'opensimplex'


In [13]:
#convert to video 
!ffmpeg -r 24 -i ./results/00001-generate-latent-walk/step%05d.png -vcodec libx264 -pix_fmt yuv420p latent-walk-v2.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [14]:
rm -r /content/drive/My Drive/stylegan2-colab-test/stylegan2/results/00002-stylegan2-birdaus-1gpu-config-f

rm: cannot remove '/content/drive/My': No such file or directory
rm: cannot remove 'Drive/stylegan2-colab-test/stylegan2/results/00002-stylegan2-birdaus-1gpu-config-f': No such file or directory
